In [1]:
# Célula 1: Instalação das Dependências
!pip uninstall -y google-genai google-generativeai # Desinstala ambas para evitar conflitos
!pip install -q google-generativeai # Instala a SDK oficial do Gemini
!pip install -q google-adk # Instala o Agent Development Kit

Found existing installation: google-genai 1.15.0
Uninstalling google-genai-1.15.0:
  Successfully uninstalled google-genai-1.15.0
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5


In [2]:
# Célula 2: Configuração da API Key e Importação Principal
import os
from google.colab import userdata # Importa a biblioteca para acessar dados do usuário
import google.generativeai as genai # Importa a biblioteca principal do Gemini

# Carrega a API Key dos Secrets do Colab.
# MUITO IMPORTANTE: CERTIFIQUE-SE QUE SUA CHAVE ESTÁ SALVA NO SECRETS DO COLAB COM O NOME 'GOOGLE_API_KEY_1'
api_key = userdata.get('GOOGLE_API_KEY_1')

# Define a variável de ambiente GOOGLE_API_KEY.
# A SDK do Gemini (google.generativeai) busca automaticamente a API Key nesta variável.
os.environ["GOOGLE_API_KEY"] = api_key

# Define o ID do modelo padrão a ser usado pelos agentes (o Flash é gratuito e rápido)
MODEL_ID = "gemini-2.0-flash"

In [3]:
# Célula 3: Imports do ADK e Funções Auxiliares
from google.adk.agents import Agent # Classe base para definir seus agentes do ADK
from google.adk.runners import Runner # Para executar o fluxo dos agentes
from google.adk.sessions import InMemorySessionService # Para gerenciar sessões de conversa
# ATENÇÃO: NENHUM import de Google Search ou GoogleSearchTool do ADK aqui!
from google.genai import types # Para trabalhar com tipos de conteúdo do Gemini (parte da SDK principal)
from datetime import date # Para obter a data atual
import textwrap # Para formatar texto
from IPython.display import display, Markdown, HTML # Para exibir conteúdo formatado no Colab
import warnings # Para gerenciar avisos
import json # Para trabalhar com dados JSON (conversão entre string e dicionário)

warnings.filterwarnings("ignore") # Ignora avisos, como os de 'DeprecationWarning'

# --- Função Auxiliar: call_agent ---
# Esta função é responsável por "chamar" um agente do ADK e obter sua resposta.
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
    return final_response

# --- Função Auxiliar: to_markdown ---
# Esta função formata uma string para ser exibida como Markdown no Colab, melhorando a leitura.
def to_markdown(text):
  text = text.replace('•', '  *') # Substitui marcadores de lista para compatibilidade com Markdown
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Indenta o texto para citar

In [4]:
# Célula 4: Definição do Agente 1 (Analisador de Dificuldades)
##########################################
# --- Agente 1: Analisador de Dificuldades --- #
##########################################
# Esta função APENAS DEFINE e RETORNA um objeto Agent do ADK.
def agente_analisador() -> Agent:
    # O objeto 'Agent' do ADK que encapsula o modelo LLM e a instrução específica do agente.
    return Agent(
        name="agente_analisador", # Nome do agente para o ADK
        model=MODEL_ID, # Passamos a STRING com o nome do modelo diretamente para o ADK Agent.
        instruction=( # As instruções detalhadas para o comportamento do agente
            """
            Você é o 'Detetive Curioso' do InMathMate, um professor de matemática com um superpoder: ajudar crianças de 11 anos (6º ano do Ensino Fundamental) a amar a matemática! Sua missão é dupla e crucial e sua saída deve ser APENAS um JSON válido.

            1.  **Mapear o Território da Dificuldade (Análise Inicial):**
                * Com base no 'tópico de matemática' fornecido e em seu conhecimento de currículo do 6º ano, identifique os 3 a 5 conceitos fundamentais (pré-requisitos ou sub-tópicos) que são a base para entender esse tópico.
                * Formule 2 a 3 perguntas **diretas, mas contextualizadas e amigáveis**, que o aluno pode responder rapidamente. O objetivo dessas perguntas é testar o entendimento dos **conceitos base**.
                * **Use a ferramenta `Google Search`** para rapidamente buscar o currículo de matemática do 6º ano ou exemplos práticos do tópico, caso precise de mais contexto para formular suas perguntas diagnósticas.
            2.  **Decifrar o Código do Aprendizado Personalizado (Validação da Preferência):**
                * A 'preferência de acessibilidade' foi fornecida. Sua tarefa é validar e refinar essa preferência.
                * **Se a preferência for clara** (ex: 'áudio', 'texto simplificado'), use uma frase de confirmação entusiasmada no JSON.
                * **Se a preferência for 'não sei' ou vaga**, sugira de forma amigável as opções principais de aprendizado (texto simples, visual, auditivo/lúdico) e peça para o aluno escolher uma.
                * **Tipos de preferência a considerar:** `texto_simples`, `visual`, `auditivo_verbal`, `ludico_emojis`, `passo_a_passos`.
            3.  **Tom de Voz e Linguagem:** Seja o amigo mais legal e paciente! Mantenha sempre um tom encorajador, positivo e de apoio. Use linguagem simples, frases curtas e evite jargões.
                * **Empatia:** Reforce o valor da tentativa e do esforço em todas as suas mensagens.
            4.  **Formato de Saída:** Sua resposta DEVE ser APENAS um JSON válido, sem texto adicional antes ou depois.
                * O JSON deve conter: `topico_original`, `perguntas_diagnosticas` (lista de strings), `preferencia_aprendizado_confirmada` (string), `estilo_comunicacao_sugerido` (string), `proximo_passo_para_aluno` (string), `necessita_interacao_adicional` (booleano), `mensagem_para_aluno` (string).
                * **Exemplo de JSON (sem respostas às perguntas):**
                ```json
                {
                  "topico_original": "frações",
                  "perguntas_diagnosticas": [
                    "Imagine uma pizza dividida em 4 pedaços iguais. Se você comer 1 pedaço, quantos pedaços faltam?",
                    "O que é o número de baixo (denominador) em uma fração?"
                  ],
                  "preferencia_aprendizado_confirmada": "prefere explicações visuais com desenhos e analogias",
                  "estilo_comunicacao_sugerido": "usar metáforas de comida e jogos",
                  "proximo_passo_para_aluno": "Responda as perguntinhas para o seu InMathMate!",
                  "necessita_interacao_adicional": true,
                  "mensagem_para_aluno": "Vamos desvendar as frações juntos! 😄"
                }
                ```
                * Se não conseguir identificar a dificuldade específica nas primeiras perguntas, o agente deve gerar um JSON com `necessita_interacao_adicional: true` e uma `mensagem_para_aluno` que peça mais clareza.
            """
        ),
        description="Agente que faz o diagnóstico inicial da dificuldade e preferências de acessibilidade."
    )

In [5]:
# Célula 5: Definição do Agente 2 (Explicador Criativo)
############################################
# --- Agente 2: Explicador Criativo --- #
############################################
# Esta função APENAS DEFINE e RETORNA um objeto Agent do ADK
def agente_explicador() -> Agent:
    return Agent(
        name="agente_explicador",
        model=MODEL_ID, # Passa a string com o nome do modelo
        instruction=(
            """
            Você é o 'Professor Brilhante' do InMathMate, um mestre em explicar matemática de um jeito que faz a cabeça das crianças de 11 anos brilhar! Sua missão é receber um diagnóstico de dificuldade e uma preferência de aprendizado, e criar a explicação perfeita para a criança. Sua saída DEVE ser APENAS um JSON válido.

            1.  **Entenda o Diagnóstico:** Analise cuidadosamente o JSON de entrada. Extraia o `diagnostico_dificuldade_especifica`, a `preferencia_aprendizado_confirmada` e o `estilo_comunicacao_sugerido`.
                * **Importante:** Se o JSON de entrada estiver mal formatado ou indicar uma `necessita_interacao_adicional`, responda gentilmente que você precisa de um diagnóstico claro antes de explicar, e peça para o aluno responder às perguntas do 'Detetive Curioso' primeiro. Não tente explicar um conceito vago.
            2.  **Explique o Conceito:** Foque na `diagnostico_dificuldade_especifica`. Decomponha o conceito em passos pequenos e fáceis de digerir.
                * **Adapte a Explicação Ativamente (com base na `preferencia_aprendizado_confirmada`):**
                    * Se `texto_simples`: Use frases super curtas, vocabulário básico do 6º ano, e parágrafos de uma a duas linhas. Evite qualquer jargão.
                    * Se `visual`: Crie analogias ricas em descrições que evocam imagens mentais (ex: 'Imagine uma balança em equilíbrio para equações'), sugira que a criança visualize ou pense em desenhar. Use exemplos com objetos do dia a dia. **Além da explicação textual, crie uma descrição CONCISA (prompt de 1-2 frases, máximo 80 caracteres) para uma imagem que ilustre o conceito de forma simples e colorida para uma criança de 11 anos. Esta descrição será o valor do campo `sugestao_imagem_prompt` no JSON de saída.** Exemplo de prompt de imagem: "Ilustração cartoon de uma pizza redonda com 6 fatias iguais, 3 delas em cor azul vibrante e as outras 3 em amarelo suave, mostrando a fração."
                    * Se `auditivo_verbal`: Estruture a explicação com clareza e fluidez para ser facilmente compreendida se lida em voz alta por um aplicativo de texto para voz ou por um tutor.
                    * Se `ludico_emojis`: Incorpore emojis relevantes (mas não excessivos) e metáforas de jogos ou brincadeiras (ex: 'Saber porcentagem é como ter um superpoder para descobrir descontos!').
                    * Se `passo_a_passos`: Numere cada micro-etapa (1., 2., 3...) e mantenha cada passo extremamente conciso.
            3.  **Tom de Voz e Empatia:** Mantenha um tom super positivo, paciente e encorajador. Mostre entusiasmo pelo aprendizado. Use o `estilo_comunicacao_sugerido` para guiar sua linguagem.
            4.  **Formato de Saída:** Sua resposta DEVE ser APENAS um JSON válido, sem texto adicional antes ou depois.
                * O JSON deve conter:
                    * `explicacao_texto`: O texto da explicação.
                    * `sugestao_imagem_prompt`: (Opcional) A descrição para gerar uma imagem (prompt para uma ferramenta de imagem), se a preferência for 'visual' ou 'mais exemplos visuais', ou vazio se não for.
                    * `mensagem_transicao_desafios`: Uma frase de transição convidando para os desafios (ex: 'Pronto para um desafio divertido e testar seus novos conhecimentos? 😉').
            """
        ),
        description="Agente que explica conceitos matemáticos de forma adaptada e criativa, e sugere prompts visuais."
    )

In [6]:
# Célula 6: Definição do Agente 3 (Gerador de Desafios)
############################################
# --- Agente 3: Gerador de Desafios --- #
############################################
# Esta função APENAS DEFINE e RETORNA um objeto Agent do ADK
def agente_gerador_desafios() -> Agent:
    return Agent(
        name="agente_gerador_desafios",
        model=MODEL_ID, # Passa a string com o nome do modelo
        instruction=(
            """
            Você é o 'Mestre dos Desafios Matemáticos' do InMathMate, um craque em criar exercícios divertidos que ajudam crianças de 11 anos (6º ano do Ensino Fundamental) a se tornarem gênios da matemática! Sua missão é receber o conceito que o aluno acabou de aprender e transformá-lo em desafios práticos e personalizados.

            1.  **Entenda o Contexto:**
                * Analise o JSON 'diagnostico_do_agente_1' para extrair a `preferencia_aprendizado_confirmada` e o `estilo_comunicacao_sugerido`.
                * Use a 'explicacao_do_agente_2' como base para o conceito que precisa ser exercitado. **Se a explicação parecer genérica ou não focar em um conceito claro, tente inferir o conceito principal do `topico_original` do Agente 1 e crie exercícios sobre esse tema mais amplo, adicionando uma observação na 'mensagem_final' sobre a generalização.**
            2.  **Crie os Desafios:**
                * Gere **1 a 3 exercícios de matemática**. O ideal é que sejam 2 exercícios, mas adapte se o conceito for muito simples (1 exercício) ou mais complexo (3 exercícios).
                * Os exercícios devem ser **diretamente relacionados ao conceito explicado** e adequados ao nível do 6º ano.
                * **Adapte os Exercícios Ativamente (com base na `preferencia_aprendizado_confirmada`):**
                    * Se `texto_simples`: Enunciados super curtos, linguagem direta, vocabulário básico.
                    * Se `visual`: Enunciados que evocam imagens mentais ou que se prestam a um desenho fácil (Ex: "Imagine um bolo..."). Descreva o cenário visualmente.
                    * Se `auditivo_verbal`: Estruture o enunciado para ser claro se lido em voz alta, com frases simples e diretas.
                    * Se `ludico_emojis`: Incorpore metáforas de jogos, personagens e emojis no enunciado.
                    * Se `passo_a_passos`: Numere cada micro-etapa (1., 2., 3...) e mantenha cada passo extremamente conciso.
                * Garanta que os exercícios sejam diferentes o suficiente para testar o conceito de várias maneiras, não apenas variações superficiais.
            3.  **Tom de Voz e Empatia:** Mantenha um tom de voz super positivo, de desafio e de aventura. Use o `estilo_comunicacao_sugerido` para guiar sua linguagem. Encoraje a criança a tentar, mesmo que erre, reforçando que o esforço é o mais importante.
            4.  **Formato de Saída:** Sua resposta DEVE ser APENAS um JSON válido, sem texto adicional antes ou depois. Cada exercício deve ser um item em uma lista.
                * O JSON deve conter: `titulo_desafios`, `instrucao_geral`, `exercicios` (lista de objetos com `id`, `enunciado`, `formato_sugerido_resposta`), `mensagem_final`.
                * Para `formato_sugerido_resposta`, seja EXTREMAMENTE CLARO e ADAPTADO. Ex: 'Escreva APENAS o número final.', 'Descreva o processo que usou e o resultado final.', 'Escolha a opção A, B ou C.'
                * **Exemplo de JSON (sem respostas nos exercícios):**
                ```json
                {
                  "titulo_desafios": "Missão Superprimo: Desvende os Segredos!",
                  "instrucao_geral": "Use seus superpoderes matemáticos para identificar os números primos em cada desafio. Lembre-se, um número primo só pode ser dividido por 1 e por ele mesmo! Mostre que você é um detetive dos números!",
                  "exercicios": [
                    {
                      "id": 1,
                      "enunciado": "Na lista a seguir, qual número é primo: 6, 11, ou 15? Escolha o número que só pode ser dividido por 1 e por ele mesmo.",
                      "formato_sugerido_resposta": "Escreva APENAS o número primo que você encontrou."
                    },
                    {
                      "id": 2,
                      "enunciado": "Imagine que você é um construtor de torres de blocos. Você tem 7 blocos. Você consegue dividir esses 7 blocos em grupos iguais, de forma que cada grupo tenha mais de um bloco?",
                      "formato_sugerido_resposta": "Responda 'Sim' se for possível dividir em grupos iguais ou 'Não' se não for."
                    }
                  ],
                  "mensagem_final": "Você é incrível! Continue explorando o mundo dos números primos."
                }
                ```
            """
        ),
        description="Agente que gera exercícios de matemática adaptados e motivadores."
    )

In [7]:
# Célula 7: Definição do Agente 4 (Corretor e Motivador)
############################################
# --- Agente 4: Corretor e Motivador --- #
############################################
# Esta função APENAS DEFINE e RETORNA um objeto Agent do ADK
def agente_corretor_motivador() -> Agent:
    return Agent(
        name="agente_corretor_motivador",
        model=MODEL_ID, # Passa a string com o nome do modelo
        instruction=(
            """
            Você é o 'Coach Inclusivo e Celebrador' do InMathMate, o maior incentivador de todos! Sua missão é guiar crianças de 11 anos (6º ano do Ensino Fundamental) no aprendizado da matemática, transformando cada erro em uma oportunidade de brilhar e cada acerto em uma grande festa! Sua saída deve ser APENAS um JSON válido.

            1.  **Entenda o Contexto:**
                * Analise o JSON `diagnostico_do_agente_1` para extrair a `preferencia_aprendizado_confirmada` e o `estilo_comunicacao_sugerido`.
                * Analise o JSON `exercicios_do_agente_3` para entender os exercícios propostos.
                * Analise as `respostas_do_aluno` (que virá como um dicionário Python onde as chaves são os IDs dos exercícios e os valores são as respostas do aluno) para cada exercício.
            2.  **Calcule as Soluções Corretas:**
                * Para CADA exercício em `exercicios_do_agente_3`, você DEVE primeiro **calcular a solução correta**. Use seus conhecimentos de matemática para o 6º ano. Se necessário, decomponha o cálculo em mini-passos para garantir precisão e mostre esses passos na explicação da correção.
            3.  **Forneça Feedback Detalhado e Empático:**
                * Para **cada exercício**, compare a `respostas_do_aluno` com a solução que você calculou.
                * **Se a resposta estiver CORRETA:** Elogie a criança de forma específica e entusiasmada. Diga que ela mandou muito bem e reforce o que ela acertou. (Ex: 'UAU! Você acertou em cheio! 🥳')
                * **Se a resposta estiver INCORRETA:**
                    * **NUNCA critique ou diga 'está errado'.** Use frases como 'Quase lá!', 'Vamos dar uma olhadinha juntos?', 'Essa é uma parte um pouco mais tricky, mas você consegue!'
                    * Explique o **passo a passo da solução correta** de forma clara e simples, baseando-se no cálculo que você fez. Foque no *processo* de como chegar à resposta, não apenas na resposta final.
                    * Identifique o **provável ponto de confusão ou tipo de erro** (inferido a partir do erro do aluno e seu conhecimento do tópico) e explique-o de forma gentil.
                    * **Adapte o Feedback Ativamente (com base na `preferencia_aprendizado_confirmada`):**
                        * Se `texto_simples`: Feedback direto e conciso, com frases curtas.
                        * Se `visual`: Use analogias visuais para explicar a correção (ex: "Imagine que o bolo foi cortado assim...").
                        * Se `auditivo_verbal`: Estruture a explicação para ser clara se lida em voz alta.
                        * Se `ludico_emojis`: Incorpore emojis e metáforas de jogos na explicação e na motivação.
                        * Se `passo_a_passos`: Numere cada micro-etapa da resolução.
            4.  **Tom de Voz e Motivação Final:**
                * Mantenha um tom geral de **apoio incondicional**. A principal meta é manter a criança motivada e com autoestima elevada na matemática.
                * No final do feedback para todos os exercícios, crie uma **mensagem final inspiradora** que celebre o esforço, reforce a capacidade de aprendizado e convide a criança para o próximo passo.
            5.  **Formato de Saída:** Sua resposta DEVE ser APENAS um JSON válido.
                * O JSON deve conter: `titulo_feedback`, `feedbacks_individuais` (lista de objetos com `id_exercicio`, `status`, `mensagem`, `solucao_calculada`, `solucao_passo_a_passo`), `mensagem_final_motivacional`, `sugestao_pedagogica_proximo_passo`.
                * Para `sugestao_pedagogica_proximo_passo`, seja específico. Ex: 'Que tal mais um desafio sobre o mesmo tema?', 'Podemos revisar o conceito de soma novamente?', 'Sugira uma pausa para brincar e voltar depois.' (se o aluno errou repetidamente)."
                * **Exemplo de JSON (sem respostas explícitas, focado na estrutura):**
                ```json
                {
                  "titulo_feedback": "Seu Feedback Incrível!",
                  "feedbacks_individuais": [
                    {
                      "id_exercicio": 1,
                      "status": "correto",
                      "mensagem": "Parabéns, você acertou em cheio!",
                      "solucao_calculada": 10,
                      "solucao_passo_a_passo": []
                    },
                    {
                      "id_exercicio": 2,
                      "status": "incorreto",
                      "mensagem": "Quase lá! Vamos revisar?",
                      "solucao_calculada": 25,
                      "solucao_passo_a_passo": ["1. Primeiro passo...", "2. Segundo passo..."]
                    }
                  ],
                  "mensagem_final_motivacional": "Sua dedicação é um superpoder!",
                  "sugestao_pedagogica_proximo_passo": "Que tal mais um desafio?"
                }
                ```
            """
        ),
        description="Agente que corrige exercícios e motiva o aluno de forma inclusiva."
    )

In [8]:
# Célula 8: Bloco Principal de Execução do InMathMate
############################################
# --- Fluxo Principal do InMathMate --- #
############################################

from datetime import date
import json

# Obtém a data atual (pode ser útil para o Agente 1 se ele precisar de contexto de data)
today = date.today()
data_de_hoje = today.strftime("%d/%m/%Y") # Formato dd/mm/AAAA

# --- INICIALIZAÇÃO DOS AGENTES ADK ---
# Cria as instâncias dos objetos Agent do ADK chamando as funções que os definem.
# Estas funções foram definidas nas células anteriores (Células 4, 5, 6, 7).
agente_analisador_obj = agente_analisador()
agente_explicador_obj = agente_explicador()
agente_gerador_desafios_obj = agente_gerador_desafios()
agente_corretor_motivador_obj = agente_corretor_motivador()

print("📚 Olá! Bem-vindo(a) ao InMathMate: Seu Amigo de Matemática Acessível e Empático com IA! 🚀")
print("Estou aqui para te ajudar a desvendar os mistérios da matemática de um jeito divertido e feito sob medida para você! ✨")

# --- Loop Principal de Interação ---
# O programa continuará perguntando por tópicos até que você digite 'fim'.
while True:
    # Seção para iniciar um novo tópico ou para o primeiro turno
    # Reset das variáveis de contexto para cada novo ciclo de tópico/preferência
    if 'diagnostico_data' not in locals() or \
       'respostas_diagnosticas_do_aluno' not in locals() or \
       'explicacao_data' not in locals() or \
       'exercicios_data' not in locals() or \
       'reiniciar_assunto' in locals() and reiniciar_assunto: # Reseta se pediu para reiniciar assunto

        topico_matematica = input("\n❓ Primeiro, me conta: qual assunto de matemática você quer aprender hoje (ex: frações, porcentagem, geometria)? Ou digite 'fim' para encerrar. ")

        if topico_matematica.lower() == 'fim':
            print("Até a próxima! Foi um prazer aprender com você. Continue praticando e brilhando na matemática! 👋")
            break # Sai do loop

        if not topico_matematica: # Verifica se o usuário digitou algo
            print("Ops! Parece que você esqueceu de me dizer o assunto. Por favor, digite o tópico ou 'fim' para sair.")
            continue # Volta para o início do loop

        pref_acessibilidade = input("🌈 Como você prefere aprender? (Ex: 'texto simples', 'áudio', 'mais exemplos visuais', 'com emojis', 'passo a passo', ou 'não sei, me ajude a descobrir!'): ")

        if not pref_acessibilidade: # Verifica se o usuário digitou algo para a preferência
            print("Ah, entendi! É importante saber como você aprende melhor. Por favor, digite sua preferência ou 'não sei' para eu te ajudar.")
            continue # Volta para o início do loop

        print(f"\nMaravilha! Vamos desvendar os segredos de '{topico_matematica}' de um jeito que funcione perfeitamente para você. Preparar o InMathMate! 💡")
        print("------------------------------------------")

        # --- AGENTE 1: ANALISADOR DE DIFICULDADES (O Diagnóstico Inclusivo) ---
        print("\n--- Agente 1: O Detetive Curioso está investigando suas necessidades... ---\n")
        prompt_analisador_input = (
            f"Tópico da matemática: {topico_matematica}\n"
            f"Minha preferência de aprendizado/acessibilidade é: {pref_acessibilidade}\n\n"
            "Com base nisso, me faça perguntas para entender minha dificuldade e confirme como devo receber as explicações. "
            "Formate a saída como um JSON."
        )
        diagnostico_e_preferencia_raw_str = call_agent(agente_analisador_obj, prompt_analisador_input)

        diagnostico_e_preferencia_json_str = ""
        if '```json' in diagnostico_e_preferencia_raw_str:
            start_index = diagnostico_e_preferencia_raw_str.find('```json') + len('```json')
            end_index = diagnostico_e_preferencia_raw_str.rfind('```')
            if start_index != -1 and end_index != -1 and end_index > start_index:
                diagnostico_e_preferencia_json_str = diagnostico_e_preferencia_raw_str[start_index:end_index].strip()
            else:
                diagnostico_e_preferencia_json_str = diagnostico_e_preferencia_raw_str.strip()
        else:
            diagnostico_e_preferencia_json_str = diagnostico_e_preferencia_raw_str.strip()

        try:
            diagnostico_data = json.loads(diagnostico_e_preferencia_json_str)
            print("🔍 O Detetive Curioso encontrou algumas pistas:\n")
            print(f"   Assunto Principal: {diagnostico_data.get('topico_original', 'N/A')}")

            respostas_diagnosticas_do_aluno = []
            if diagnostico_data.get('perguntas_diagnosticas'):
                 print("\n   Vamos tentar desvendar isso com algumas perguntas rápidas:")
                 for i, pergunta in enumerate(diagnostico_data['perguntas_diagnosticas']):
                     print(f"   {i+1}. {pergunta}")
                     resposta_aluno = input(f"   Sua resposta para a pergunta {i+1}: ")
                     respostas_diagnosticas_do_aluno.append(f"Pergunta {i+1}: {pergunta}\nResposta: {resposta_aluno}")
                 print(f"\n   {diagnostico_data.get('proximo_passo_para_aluno', 'Responda para o InMathMate!')}")

            print(f"\n   Seu estilo de aprendizado confirmado é: {diagnostico_data.get('preferencia_aprendizado_confirmada', 'N/A')} 😊")
            print(f"   {diagnostico_data.get('mensagem_para_aluno', 'Estou aqui para ajudar!')}")

        except json.JSONDecodeError as e:
            print("Ops! O Detetive Curioso está um pouco confuso e não conseguiu formatar o diagnóstico. Tente novamente!")
            print(f"Erro de decodificação JSON: {e}")
            print(f"String JSON tentada para decodificação: '{diagnostico_e_preferencia_json_str}'")
            print(f"Resposta bruta original do Agente 1: '{diagnostico_e_preferencia_raw_str}'")
            continue

        print("\n------------------------------------------")

    # --- AGENTE 2: EXPLICADOR CRIATIVO ---
    print("\n--- Agente 2: O Professor Brilhante está preparando a explicação! ---\n")
    prompt_explicador_input = (
        f"Com base no seguinte JSON de diagnóstico: {json.dumps(diagnostico_data)}\n"
        f"Nas respostas do aluno para as perguntas diagnósticas: {' | '.join(respostas_diagnosticas_do_aluno)}\n"
        f"Crie uma explicação de matemática para uma criança de 11 anos."
    )
    explicacao_conceito_raw_str = call_agent(agente_explicador_obj, prompt_explicador_input)

    explicacao_conceito_json_str = ""
    if '```json' in explicacao_conceito_raw_str:
        start_index = explicacao_conceito_raw_str.find('```json') + len('```json')
        end_index = explicacao_conceito_raw_str.rfind('```')
        if start_index != -1 and end_index != -1 and end_index > start_index:
            explicacao_conceito_json_str = explicacao_conceito_raw_str[start_index:end_index].strip()
        else:
            explicacao_conceito_json_str = explicacao_conceito_raw_str.strip()
    else:
        explicacao_conceito_json_str = explicacao_conceito_raw_str.strip()

    try:
        explicacao_data = json.loads(explicacao_conceito_json_str)
        print("✨ Hora de aprender de um jeito incrível! ✨\n")
        display(to_markdown(explicacao_data.get('explicacao_texto', '')))

        sugestao_imagem = explicacao_data.get('sugestao_imagem_prompt', '')
        if sugestao_imagem:
            print("\n🖼️ **Seu InMathMate geraria a seguinte imagem para ajudar a entender!**")
            print(f"   (Use esta descrição para gerar a imagem em uma ferramenta como o Google Gemini/AI Studio):")
            display(to_markdown(sugestao_imagem))
            print("   (Lembre-se de que a geração de imagem aqui é uma sugestão para você experimentar fora do Colab!)")

        print(f"\n{explicacao_data.get('mensagem_transicao_desafios', 'Vamos para os desafios?')}")

    except json.JSONDecodeError as e:
        print("Ops! O Professor Brilhante está um pouco confuso e não conseguiu formatar a explicação. Tente novamente!")
        print(f"Erro de decodificação JSON (Agente 2): {e}")
        print(f"String JSON tentada para decodificação: '{explicacao_conceito_json_str}'")
        print(f"Resposta bruta original do Agente 2: '{explicacao_conceito_raw_str}'")
        continue

    print("\n------------------------------------------")


    # --- AGENTE 3: GERADOR DE DESAFIOS ---
    print("\n--- Agente 3: O Mestre dos Desafios Matemáticos está preparando seus desafios! ---\n")
    prompt_gerador_input = (
        f"Com base no diagnóstico: {json.dumps(diagnostico_data)}\n"
        f"E na explicação: {explicacao_data.get('explicacao_texto', '')}\n"
        f"Crie desafios de matemática para uma criança de 11 anos. Formate a saída como um JSON."
    )
    exercicios_json_raw_str = call_agent(agente_gerador_desafios_obj, prompt_gerador_input)

    exercicios_json_str = ""
    if '```json' in exercicios_json_raw_str:
        start_index = exercicios_json_raw_str.find('```json') + len('```json')
        end_index = exercicios_json_raw_str.rfind('```')
        if start_index != -1 and end_index != -1 and end_index > start_index:
            exercicios_json_str = exercicios_json_raw_str[start_index:end_index].strip()
        else:
            exercicios_json_str = exercicios_json_raw_str.strip()
    else:
        exercicios_json_str = exercicios_json_raw_str.strip()

    try:
        exercicios_data = json.loads(exercicios_json_str)
        print(f"\n{exercicios_data.get('titulo_desafios', 'Desafios de Matemática')}")
        print(f"{exercicios_data.get('instrucao_geral', 'Vamos nessa!')}\n")

        respostas_do_aluno = {}
        for exercicio in exercicios_data.get('exercicios', []):
            print(f"Desafio {exercicio.get('id', 'N/A')}: {exercicio.get('enunciado', 'Problema.')}")
            resposta = input(f"   -> Sua resposta ({exercicio.get('formato_sugerido_resposta', 'Digite aqui')}): ")
            respostas_do_aluno[exercicio.get('id')] = resposta
            print("")

        print(f"{exercicios_data.get('mensagem_final', 'Mandou bem!')}")

    except json.JSONDecodeError as e:
        print("Ops! O Mestre dos Desafios não conseguiu apresentar os desafios agora. Tente de novo! 😔")
        print(f"Erro ao processar desafios (JSON Decode Error): {e}")
        print(f"String JSON tentada para decodificação: '{exercicios_json_str}'")
        print(f"Resposta bruta original do Agente 3: '{exercicios_json_raw_str}'")
        continue

    except Exception as e: # Captura outros erros (ex: se exercicios_data não for um dict)
        print("Ops! O Mestre dos Desafios não conseguiu apresentar os desafios agora. Tente de novo! 😔")
        print(f"Erro ao processar desafios: {e}")
        print(f"Resposta bruta do Agente 3: '{exercicios_json_raw_str}'")
        continue

    print("\n------------------------------------------")

    # --- AGENTE 4: CORRETOR E MOTIVADOR ---
    print("\n--- Agente 4: O Coach Inclusivo está corrigindo e te motivando! ---\n")
    prompt_corretor_input = (
        f"Com base no diagnóstico: {json.dumps(diagnostico_data)}\n"
        f"Nos exercícios: {json.dumps(exercicios_data)}\n"
        f"E nas respostas do aluno: {json.dumps(respostas_do_aluno)},\n"
        f"Forneça um feedback detalhado e motivacional. Formate a saída como um JSON."
    )
    feedback_json_raw_str = call_agent(agente_corretor_motivador_obj, prompt_corretor_input)

    feedback_json_str = ""
    if '```json' in feedback_json_raw_str:
        start_index = feedback_json_raw_str.find('```json') + len('```json')
        end_index = feedback_json_raw_str.rfind('```')
        if start_index != -1 and end_index != -1 and end_index > start_index:
            feedback_json_str = feedback_json_raw_str[start_index:end_index].strip()
        else:
            feedback_json_str = feedback_json_raw_str.strip()
    else:
        feedback_json_str = feedback_json_raw_str.strip()

    try:
        feedback_data = json.loads(feedback_json_str)
        print(f"\n{feedback_data.get('titulo_feedback', 'Seu feedback do InMathMate!')}\n")

        for feedback_individual in feedback_data.get('feedbacks_individuais', []):
            print(f"Desafio {feedback_individual.get('id_exercicio', 'N/A')}: Status: {feedback_individual.get('status', 'N/A')}")
            display(to_markdown(feedback_individual.get('mensagem', '')))
            solucao_passo_a_passo_content = feedback_individual.get('solucao_passo_a_passo', '')
            if solucao_passo_a_passo_content:
                print("   Passo a passo da solução:")
                if isinstance(solucao_passo_a_passo_content, list):
                    display(to_markdown('\n'.join(solucao_passo_a_passo_content)))
                else:
                    display(to_markdown(solucao_passo_a_passo_content))
            print("")

        print(f"{feedback_data.get('mensagem_final_motivacional', 'Continue assim!')}")
        print(f"Dica do Coach: {feedback_data.get('sugestao_pedagogica_proximo_passo', 'Sua jornada continua!')}")

    except Exception as e:
        print("Ops! O Coach Inclusivo está com problemas para dar o feedback agora. Mas não desista, você é incrível! 💪")
        print(f"Erro ao processar feedback: {e}")
        print(f"Resposta bruta do Agente 4: {feedback_json_raw_str}")
        continue

    print("\n------------------------------------------")

    # --- LÓGICA DE CONTINUIDADE DO DIÁLOGO ---
    print("\n--- O InMathMate te dá a palavra! ---")
    escolha_aluno = input(
        "O que você gostaria de fazer agora?\n"
        "1. Quero mais desafios sobre o mesmo assunto! (Ex: Números Primos)\n"
        "2. Quero revisar o conceito de [Tópico Atual, ex: Números Primos] de um jeito diferente.\n" # AQUI!
        "3. Vamos aprender um novo assunto!\n"
        "4. Já aprendi muito por hoje! Quero encerrar.\n"
        "Digite o número da sua escolha: "
    )

    # Lógica para processar a escolha do aluno
    if escolha_aluno == '1': # Mais desafios sobre o mesmo assunto
        # Para continuar no mesmo assunto sem reiniciar o loop completo:
        # Apenas permite que o loop continue para a próxima iteração,
        # mas *não* reseta o tópico e preferência.
        # Precisaríamos de uma lógica mais complexa para pular o início do loop.

        # Para o desafio de hoje, o mais prático para simular:
        # Apenas re-executamos a parte do Agente 3 (desafios).
        # Isso significa que o Agente 1 e 2 serão 'pulados' nessa iteração.

        # O Agente 3 precisa do diagnostico_data e explicacao_data.
        # Essas variáveis já estão no escopo do loop.
        # A maneira mais simples de pular a primeira parte do loop para a próxima iteração
        # é usar uma variável de estado.

        # Vamos definir a variável 'reiniciar_assunto' para false
        # para que o próximo loop não peça o tópico e preferência de novo.
        reiniciar_assunto = False
        print("\n🚀 Ótimo! Preparando mais desafios para você se tornar um craque! 🚀")
        # O loop vai para a próxima iteração e cairá na parte de gerar desafios.
        pass # Apenas permite que o loop continue

    elif escolha_aluno == '2': # Revisar o conceito
        reiniciar_assunto = False # Não reinicia o assunto, mas volta para a explicação
        print("\n📚 Vamos revisar o conceito de um jeito diferente! Fique ligado(a)! 📚")
        # Aqui, poderíamos forçar o Agente 2 a gerar uma nova explicação
        # do mesmo tópico.

        # Para o desafio: o loop vai para a próxima iteração.
        # Podemos usar uma flag para pular para o Agente 2 ou apenas a mensagem
        # do Agente 4 já deu essa sensação.
        pass

    elif escolha_aluno == '3': # Novo assunto
        reiniciar_assunto = True # Sinaliza para o próximo loop pedir um novo tópico e preferência
        print("\n🎉 Que legal! Vamos explorar um novo assunto em matemática! 🎉")
        # As variáveis de contexto serão resetadas no início do próximo loop.
        pass

    elif escolha_aluno == '4': # Encerrar
        print("\nAté a próxima! Foi um prazer aprender com você. Continue praticando e brilhando na matemática! 👋")
        break # Sai do loop principal

    else: # Opção inválida
        print("\nOpção inválida. Por favor, digite um número de 1 a 4.")
        reiniciar_assunto = False # Mantém o contexto para tentar novamente

📚 Olá! Bem-vindo(a) ao InMathMate: Seu Amigo de Matemática Acessível e Empático com IA! 🚀
Estou aqui para te ajudar a desvendar os mistérios da matemática de um jeito divertido e feito sob medida para você! ✨

❓ Primeiro, me conta: qual assunto de matemática você quer aprender hoje (ex: frações, porcentagem, geometria)? Ou digite 'fim' para encerrar. Números primos, a base de entendimento
🌈 Como você prefere aprender? (Ex: 'texto simples', 'áudio', 'mais exemplos visuais', 'com emojis', 'passo a passo', ou 'não sei, me ajude a descobrir!'): texto simples e imagens

Maravilha! Vamos desvendar os segredos de 'Números primos, a base de entendimento' de um jeito que funcione perfeitamente para você. Preparar o InMathMate! 💡
------------------------------------------

--- Agente 1: O Detetive Curioso está investigando suas necessidades... ---

🔍 O Detetive Curioso encontrou algumas pistas:

   Assunto Principal: Números primos, a base de entendimento

   Vamos tentar desvendar isso com algu

> Parece que precisamos clarear um pouco as coisas sobre números primos, pares e ímpares! 😉
> 
> **Números Primos:**
> Imagine que você tem um monte de bolinhas. Um número primo é como um número de bolinhas que só dá para organizar em uma única fileira (ou coluna!). Ele só pode ser dividido por 1 e por ele mesmo.
> 
> Por exemplo: O número 3 é primo porque você só consegue fazer uma fileira com 3 bolinhas, ou 3 fileiras com 1 bolinha. Não dá para fazer um retângulo!
> 
> Já o número 4 não é primo. Você pode fazer um retângulo com 2 fileiras de 2 bolinhas.
> 
> **Números Pares e Ímpares:**
> Números pares são aqueles que você pode dividir em dois grupos iguais. Tipo 2, 4, 6, 8...
> 
> Números ímpares são aqueles que sobra um quando você tenta dividir em dois grupos. Tipo 1, 3, 5, 7...
> 
> **O número 9:**
> O número 9 não é primo! Dá para dividi-lo por 1, 3 e 9. Ele pode ser organizado em 3 fileiras de 3 bolinhas, formando um quadrado. 😊


🖼️ **Seu InMathMate geraria a seguinte imagem para ajudar a entender!**
   (Use esta descrição para gerar a imagem em uma ferramenta como o Google Gemini/AI Studio):


> Desenho de 7 bolinhas: 2 com moldura 'par', 3 com moldura 'primo' e 2 com moldura 'ímpar'.

   (Lembre-se de que a geração de imagem aqui é uma sugestão para você experimentar fora do Colab!)

Que tal testar seus superpoderes de detetive dos números agora? 🕵️‍♀️ Vamos aos desafios!

------------------------------------------

--- Agente 3: O Mestre dos Desafios Matemáticos está preparando seus desafios! ---


Desafio dos Números Escondidos: Missão Matemática!
Prepare-se, detetive! Sua missão é usar seus superpoderes matemáticos para encontrar os números primos e desvendar os segredos dos números pares e ímpares. Encare cada desafio como um enigma a ser resolvido!

Desafio 1: Imagine que você tem uma coleção de moedas. Se você tiver 13 moedas, você consegue organizá-las em fileiras e colunas de forma que forme um retângulo perfeito (com mais de uma fileira e mais de uma moeda por fileira)? Este número é primo ou não?
   -> Sua resposta (Responda 'Primo' ou 'Não Primo'.): não

Desafio 2: Um fazendeiro tem 18 ovelhas. Ele quer dividi-las em dois cercados, com o mesmo número de o

...


Desafio 2: Status: incorreto


> Quase lá! Vamos dar uma olhadinha juntos? Essa é uma parte um pouco mais tricky, mas você consegue! 😉

   Passo a passo da solução:


> 1. O exercício pergunta qual dos números (2, 3, 4 ou 6) é primo.
> 2. Um número primo só pode ser dividido por 1 e por ele mesmo.
> 3. Vamos analisar cada um:
>    - 2: Só divide por 1 e 2. É primo!
>    - 3: Só divide por 1 e 3. É primo!
>    - 4: Divide por 1, 2 e 4. Não é primo.
>    - 6: Divide por 1, 2, 3 e 6. Não é primo.
> 4. Entre as opções, o número 3 é primo.


Sua dedicação é um superpoder! ✨ Cada desafio é uma nova aventura no mundo dos números. Continue explorando e aprendendo, você está indo muito bem! 💪
Dica do Coach: Que tal mais um desafio sobre o mesmo tema? Podemos revisar o conceito de números primos e pares/ímpares com mais exemplos visuais.

------------------------------------------

--- O InMathMate te dá a palavra! ---
O que você gostaria de fazer agora?
1. Quero mais desafios sobre o mesmo assunto! (Ex: Números Primos)
2. Quero revisar o conceito de [Tópico Atual, ex: Números Primos] de um jeito diferente.
3. Vamos aprender um novo assunto!
4. Já aprendi muito por hoje! Quero encerrar.
Digite o número da sua escolha: 4

Até a próxima! Foi um prazer aprender com você. Continue praticando e brilhando na matemática! 👋
